In [2]:
url="https://s3.us-east-2.amazonaws.com/datafaculty/bootcamp/HR_comma_sep.csv"

In [4]:
import pandas as pd
import numpy as np

In [5]:
data=pd.read_csv(url)

In [6]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [8]:
data.isnull().sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
sales                    0
salary                   0
dtype: int64

In [10]:
data=pd.get_dummies(data)

In [11]:
### Train, test split
import sklearn.model_selection as model_selection

In [12]:
y=data['left']
X=data.drop('left',axis=1)

In [14]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,
                                test_size=0.20,
                                random_state=200)

In [18]:
### Model
import sklearn.ensemble as ensemble

In [16]:
clf=ensemble.RandomForestClassifier(n_estimators=100,
                                   oob_score=True,n_jobs=-1)

In [19]:
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [20]:
clf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [21]:
clf.predict_proba(X_test)[:,1]

array([0.  , 0.  , 0.  , ..., 0.  , 0.14, 0.99])

In [22]:
pd.Series(clf.feature_importances_,
          index=X_train.columns).sort_values(ascending=False)

satisfaction_level       0.309919
time_spend_company       0.180761
number_project           0.172756
average_montly_hours     0.164632
last_evaluation          0.123617
Work_accident            0.010277
salary_low               0.006768
salary_high              0.005202
salary_medium            0.003841
sales_technical          0.003800
sales_sales              0.003757
sales_support            0.002959
sales_RandD              0.001756
sales_IT                 0.001599
sales_management         0.001495
promotion_last_5years    0.001488
sales_marketing          0.001487
sales_hr                 0.001485
sales_accounting         0.001317
sales_product_mng        0.001085
dtype: float64

In [23]:
clf.oob_score_

0.9895824652054338

In [24]:
## If 100, 200 or 300 trees is better
for i in [100,200,300]:
    clf=ensemble.RandomForestClassifier(n_estimators=i,
                                       oob_score=True,
                                       n_jobs=-1)
    clf.fit(X_train,y_train)
    print("oob score for {} trees is {}".format(i,clf.oob_score_))

oob score for 100 trees is 0.9899158263188599
oob score for 200 trees is 0.9904992082673556
oob score for 300 trees is 0.9905825485457121


In [25]:
### get prob predictions on oob data 
pred_oob=clf.oob_decision_function_[:,1]

In [27]:
import sklearn.metrics as metrics

In [28]:
metrics.roc_auc_score(y_train,pred_oob)

0.9928028325943286

In [29]:
## If 100, 200 or 300 trees is better ## We can also use a range of n_estimators to check for different oob Score.

for i in [100,200,300]:
    clf=ensemble.RandomForestClassifier(n_estimators=i,
                                       oob_score=True,
                                       n_jobs=-1)
    clf.fit(X_train,y_train)
    pred_oob=clf.oob_decision_function_[:,1]
    auc=metrics.roc_auc_score(y_train,pred_oob)
    print("Auc for {} trees is {}".format(i,auc))

Auc for 100 trees is 0.9923275258610372
Auc for 200 trees is 0.9930713756729351
Auc for 300 trees is 0.9930034802408809


In [31]:
# 200 tress is better
# Remodel using 200 trees

clf1=ensemble.RandomForestClassifier(n_estimators=200,
                                   oob_score=True,n_jobs=-1)


In [33]:
clf1.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [35]:
clf1.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [37]:
clf1.predict_proba(X_test)[:,1]

array([0.   , 0.   , 0.005, ..., 0.   , 0.185, 1.   ])

In [39]:
pd.Series(clf1.feature_importances_,
          index=X_train.columns).sort_values(ascending=False)

satisfaction_level       0.313849
number_project           0.186301
time_spend_company       0.170685
average_montly_hours     0.151013
last_evaluation          0.128431
Work_accident            0.010740
salary_low               0.006481
salary_high              0.005674
sales_technical          0.003962
salary_medium            0.003885
sales_sales              0.003727
sales_support            0.002980
promotion_last_5years    0.001759
sales_IT                 0.001732
sales_RandD              0.001650
sales_accounting         0.001572
sales_marketing          0.001519
sales_management         0.001433
sales_hr                 0.001415
sales_product_mng        0.001191
dtype: float64

In [51]:
clf1.oob_score_

0.9905825485457121

In [52]:
pred_oob=clf1.oob_decision_function_[:,1]

In [54]:

metrics.roc_auc_score(y_train,pred_oob)

0.9927115956881989